First create another container (Gold layer) on your adls and create external location on databricks 

You need to add a storage path to your catalog creation command — something like this:

In [0]:
# Create Catalog and Schema
spark.sql("""
CREATE CATALOG IF NOT EXISTS safe_sure_catalog
MANAGED LOCATION 'abfss://gold-layer@sampledemos.dfs.core.windows.net/'
""")

# Switch to the Catalog
spark.sql("USE CATALOG safe_sure_catalog")

# Create Schema (use either DATABASE or SCHEMA — they are synonyms in Databricks)
spark.sql("CREATE SCHEMA IF NOT EXISTS gold")
spark.sql("USE gold")


Create the agent Delta table

In [0]:
%sql
USE CATALOG safe_sure_catalog;
USE SCHEMA gold;

CREATE OR REPLACE TABLE transaction (
    TransactionID STRING,
    PolicyNo STRING,
    CustomerID STRING,
    ProductID STRING,
    TransactionDate DATE,
    Amount DECIMAL(18,2),
    PaymentMethod STRING,
    Channel STRING
)
USING DELTA;


In [0]:
# load the data from the silver layer 

df = spark.read.option("header", "true").csv("abfss://landingzone@sampledemos.dfs.core.windows.net/Transaction.csv")

from pyspark.sql import functions as F


df_transaction_clean = (
    df
    .withColumn("TransactionDate", F.to_date(F.col("TransactionDate"), "yyyy/MM/dd"))
    .withColumn("Amount", F.regexp_replace("Amount", "[₦,]", "").cast("double"))
    .withColumn("PaymentMethod", F.initcap(F.trim(F.col("PaymentMethod"))))
    .withColumn("Channel", F.initcap(F.trim(F.col("Channel"))))
    # rename to avoid Spark confusing columns
    .select(
        F.col("TransactionID").alias("TransactionID"),
        F.col("PolicyNo").alias("PolicyNo"),
        F.col("CustomerID").alias("CustomerID"),
        F.col("ProductID").alias("ProductID"),
        F.col("TransactionDate").alias("TransactionDate"),
        F.col("Amount").alias("Amount"),
        F.col("PaymentMethod").alias("PaymentMethod"),
        F.col("Channel").alias("Channel")
    )
)

df_transaction_clean.write.mode("overwrite").option("overwriteSchema", "true").saveAsTable("gold.transaction")






In [0]:
%sql
select * from gold.transaction